train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV, RandomizerSearchCV

### Model Selection

### Cross validation

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting**. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a **test set** `X_test`, `y_test`.

In scikit-learn a random split into training and test sets can be quickly computed with the `train_test_split` helper function. Let’s load the iris data set to fit a linear support vector machine on it:

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import linear_model

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

We can now quickly sample a training set while holding out 40% of the data for testing (evaluating) our classifier:

In [2]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(90, 4) (90,)
(60, 4) (60,)


In [8]:
clf = linear_model.LogisticRegression(C=100, max_iter=10000).fit(X_train, y_train)
clf.score(X_test, y_test)

0.95

When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be manually set for an LogisticRegression, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.

A solution to this problem is a procedure called **cross-validation** (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:
- A model is trained using $k-1$ of the folds as training data;
- the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data.

<div style="display:flex;justify-content:center;align-items:center;">
    <img src="images/k-fold-cross-validation.png" width="700" />
</div>

The simplest way to use cross-validation is to call the `cross_val_score` helper function on the estimator and the dataset.

The following example demonstrates how to estimate the accuracy of a logistic regression on the iris dataset by splitting the data, fitting a model and computing the score 5 consecutive times (with different splits each time):

In [12]:
from sklearn.model_selection import cross_val_score
clf = linear_model.LogisticRegression(C=10, max_iter=10000)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.96666667, 1.        , 0.96666667, 0.93333333, 1.        ])

In [13]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.97 accuracy with a standard deviation of 0.02


By default, the score computed at each CV iteration is the score method of the estimator. It is possible to change this by using the scoring parameter:

In [24]:
clf = linear_model.LogisticRegression(C=10, max_iter=10000)
scores = cross_val_score(clf, X, y, cv=5, scoring='precision_macro')
scores

array([0.96969697, 1.        , 0.96969697, 0.93333333, 1.        ])

#### The `cross_validate` function and multiple metric evaluation

The `cross_validate` function differs from `cross_val_score` in two ways:

- It allows specifying multiple metrics for evaluation.
- It returns a dict containing fit-times, score-times (and optionally training scores, fitted estimators, train-test split indices) in addition to the test score.

In [27]:
from sklearn.model_selection import cross_validate
scoring = ['precision_macro', 'recall_macro']
clf = linear_model.LogisticRegression(max_iter=1000)
scores = cross_validate(clf, X, y, scoring=scoring)
sorted(scores.keys())

['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']

In [28]:
scores['test_recall_macro']

array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ])

### Obtaining predictions by cross-validation

The function `cross_val_predict` has a similar interface to `cross_val_score`, but returns, for each element in the input, the prediction that was obtained for that element when it was in the test set. Only cross-validation strategies that assign all elements to a test set exactly once can be used (otherwise, an exception is raised).

In [34]:
from sklearn.model_selection import cross_val_predict
clf = linear_model.LogisticRegression(max_iter=1000)
predictions = cross_val_predict(clf, X, y)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

> Note on inappropriate usage of cross_val_predict.
>
> The result of `cross_val_predict` may be different from those obtained using `cross_val_score` as the elements are grouped in different ways. The function `cross_val_score` takes an average over cross-validation folds, whereas `cross_val_predict` simply returns the labels (or probabilities) from several distinct models undistinguished. Thus, `cross_val_predict` is not an appropriate measure of generalization error.

The function `cross_val_predict` is appropriate for:
- Visualization of predictions obtained from different models.
- Model blending: When predictions of one supervised estimator are used to train another estimator in ensemble methods.

### What is `KFold`?

`KFold` is a **data-splitting strategy** — not a scoring or model evaluation function.

Think of `KFold` as just a **smart iterator** that gives you **train/test indices** to use for cross-validation.

It does **not train a model** or **evaluate scores** — that’s up to you.

So in short:
- Use `KFold` when you want **custom control** over training, validation, logging, saving models, metrics, etc.
- Use `cross_val_score` or `cross_val_predict` when you want **quick, automated** results.

In [37]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)

    print(f"Fold {fold}: Accurady = {acc:.4f}")
    accuracies.append(acc)

Fold 1: Accurady = 1.0000
Fold 2: Accurady = 1.0000
Fold 3: Accurady = 0.9333
Fold 4: Accurady = 0.9667
Fold 5: Accurady = 0.9667


In [39]:
print(f"\nAverage Accuracy: {np.mean(accuracies):.4f}")


Average Accuracy: 0.9733


In [40]:
from sklearn.model_selection import LeaveOneOut

a = [45, 76, 34, 32, 21]
loo = LeaveOneOut()
for train, test in loo.split(a):
    print(train, test)

[1 2 3 4] [0]
[0 2 3 4] [1]
[0 1 3 4] [2]
[0 1 2 4] [3]
[0 1 2 3] [4]


In [45]:
from sklearn.model_selection import RepeatedKFold

a = [45, 76, 34, 32, 21, 65, 43, 32, 21]
rkf = RepeatedKFold(n_splits=4, n_repeats=3, random_state=42)

for train, test in rkf.split(a):
    print(train, test)

[0 2 3 4 6 8] [1 5 7]
[1 2 3 4 5 6 7] [0 8]
[0 1 3 5 6 7 8] [2 4]
[0 1 2 4 5 7 8] [3 6]
[2 3 4 6 7 8] [0 1 5]
[0 1 2 5 6 7 8] [3 4]
[0 1 2 3 4 5 6] [7 8]
[0 1 3 4 5 7 8] [2 6]
[1 3 4 5 6 7] [0 2 8]
[0 1 2 3 4 5 8] [6 7]
[0 1 2 4 6 7 8] [3 5]
[0 2 3 5 6 7 8] [1 4]


In [49]:
from sklearn.model_selection import LeavePOut

a = [45, 76, 34, 32, 21, 65, 43, 32, 21]
lpo = LeavePOut(p=2)

for train, test in lpo.split(a):
    print(train, test)

[2 3 4 5 6 7 8] [0 1]
[1 3 4 5 6 7 8] [0 2]
[1 2 4 5 6 7 8] [0 3]
[1 2 3 5 6 7 8] [0 4]
[1 2 3 4 6 7 8] [0 5]
[1 2 3 4 5 7 8] [0 6]
[1 2 3 4 5 6 8] [0 7]
[1 2 3 4 5 6 7] [0 8]
[0 3 4 5 6 7 8] [1 2]
[0 2 4 5 6 7 8] [1 3]
[0 2 3 5 6 7 8] [1 4]
[0 2 3 4 6 7 8] [1 5]
[0 2 3 4 5 7 8] [1 6]
[0 2 3 4 5 6 8] [1 7]
[0 2 3 4 5 6 7] [1 8]
[0 1 4 5 6 7 8] [2 3]
[0 1 3 5 6 7 8] [2 4]
[0 1 3 4 6 7 8] [2 5]
[0 1 3 4 5 7 8] [2 6]
[0 1 3 4 5 6 8] [2 7]
[0 1 3 4 5 6 7] [2 8]
[0 1 2 5 6 7 8] [3 4]
[0 1 2 4 6 7 8] [3 5]
[0 1 2 4 5 7 8] [3 6]
[0 1 2 4 5 6 8] [3 7]
[0 1 2 4 5 6 7] [3 8]
[0 1 2 3 6 7 8] [4 5]
[0 1 2 3 5 7 8] [4 6]
[0 1 2 3 5 6 8] [4 7]
[0 1 2 3 5 6 7] [4 8]
[0 1 2 3 4 7 8] [5 6]
[0 1 2 3 4 6 8] [5 7]
[0 1 2 3 4 6 7] [5 8]
[0 1 2 3 4 5 8] [6 7]
[0 1 2 3 4 5 7] [6 8]
[0 1 2 3 4 5 6] [7 8]


In [50]:
from sklearn.model_selection import ShuffleSplit
X = np.arange(10)
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_index, test_index in ss.split(X):
    print("%s %s" % (train_index, test_index))

[9 1 6 7 3 0 5] [2 8 4]
[2 9 8 0 6 7 4] [3 5 1]
[4 5 1 0 6 9 7] [2 3 8]
[2 7 5 8 0 3 4] [6 1 9]
[4 1 0 6 8 9 3] [5 2 7]


**Stratified k-fold**

In [51]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print('train -  {}   |   test -  {}'.format(
        np.bincount(y[train]), np.bincount(y[test])))
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
    print('train -  {}   |   test -  {}'.format(
        np.bincount(y[train]), np.bincount(y[test])))

train -  [30  3]   |   test -  [15  2]
train -  [30  3]   |   test -  [15  2]
train -  [30  4]   |   test -  [15  1]
train -  [28  5]   |   test -  [17]
train -  [28  5]   |   test -  [17]
train -  [34]   |   test -  [11  5]


In [52]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)
for train, test in tscv.split(X):
    print("%s %s" % (train, test))

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]


In [53]:
clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}